In [3]:
!pip install transformers==4.44.2 joblib==1.4.2 scikit-learn==1.6.0 numpy==1.26.4 pandas==2.2.3 scipy==1.13.1 seaborn==0.13.2 tqdm==4.66.5 lightgbm==4.5.0 xgboost==2.1.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 74.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 84.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 44.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 10.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 81.1 MB/s eta 0:00:00:00:01
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.67.1
    U

In [4]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [5]:
# Load datasets
train_df = pd.read_csv('/kaggle/input/caco2-set/Train_Caco2.csv')
train_df = train_df[['ID', 'SMILES', 'Permeability']]
test_df = pd.read_csv('/kaggle/input/caco2-set/Test_Caco2.csv')
test_df = test_df[['ID', 'SMILES', 'Permeability']]

In [6]:
tokenizer = AutoTokenizer.from_pretrained('seyonec/ChemBERTa-zinc-base-v1')
model = AutoModelForSequenceClassification.from_pretrained('seyonec/ChemBERTa-zinc-base-v1', num_labels=1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Custom dataset class
class SMILESDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=325):
        self.tokenizer = tokenizer
        self.dataframe = dataframe
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        smiles = self.dataframe.iloc[idx]['SMILES']
        permeability = self.dataframe.iloc[idx]['Permeability']
        inputs = self.tokenizer(smiles, return_tensors='pt', padding="max_length", truncation=True, max_length=self.max_length)
        
        input_ids = inputs['input_ids'].squeeze(0)  # Shape: (sequence_length,)
        attention_mask = inputs['attention_mask'].squeeze(0)  # Shape: (sequence_length,)
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(permeability, dtype=torch.float)
        }


# datasets
train_dataset = SMILESDataset(train_df, tokenizer)
test_dataset = SMILESDataset(test_df, tokenizer)
batch_size = 16
# data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

tokenizer_config.json:   0%|          | 0.00/166 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/501 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/179M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at seyonec/ChemBERTa-zinc-base-v1 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
num_epochs = 20

In [8]:
# Training loop
from tqdm import tqdm
for epoch in range(num_epochs):
    print(f"Entered Epoch {epoch + 1}")
    model.train()
    train_loss = 0

    for batch in tqdm(train_loader, desc=f'Training Epoch {epoch + 1}/{num_epochs}', unit='batch'):
        optimizer.zero_grad()

        # Move all batch tensors to device
        batch = {k: v.to(device) for k, v in batch.items()}
        labels = batch["labels"].unsqueeze(1)  # still shape: (batch_size, 1)

        # Forward pass
        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=labels
        )
        loss = outputs.loss
        train_loss += loss.item()

        # Backprop and optimizer step
        loss.backward()
        optimizer.step()

    avg_train_loss = train_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{num_epochs} - Train Loss: {avg_train_loss:.4f}')

Entered Epoch 1


Training Epoch 1/20: 100%|██████████| 63/63 [00:27<00:00,  2.28batch/s]


Epoch 1/20 - Train Loss: 2.1308
Entered Epoch 2


Training Epoch 2/20: 100%|██████████| 63/63 [00:28<00:00,  2.24batch/s]


Epoch 2/20 - Train Loss: 0.5081
Entered Epoch 3


...

Epoch 18/20 - Train Loss: 0.0886
Entered Epoch 19


Training Epoch 19/20: 100%|██████████| 63/63 [00:32<00:00,  1.93batch/s]


Epoch 19/20 - Train Loss: 0.0990
Entered Epoch 20


Training Epoch 20/20: 100%|██████████| 63/63 [00:31<00:00,  1.97batch/s]

Epoch 20/20 - Train Loss: 0.0900


In [9]:
model_name = 'ChemBERTa_model_1_caco2'
model_save_path = f'/kaggle/working/{model_name}'
os.makedirs(model_save_path, exist_ok=True)

tokenizer.save_pretrained(model_save_path)
model.save_pretrained(model_save_path)

print(f'Model and tokenizer saved to {model_save_path}')

Model and tokenizer saved to /kaggle/working/ChemBERTa_model_1_caco2


In [10]:
from scipy.stats import pearsonr, spearmanr

model.eval()
test_loss = 0
test_true_labels = []
predictions = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc='Testing', unit='batch'):
      
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].unsqueeze(1).to(device).float()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        test_loss += loss.item()

        test_true_labels.extend(labels.cpu().numpy())
        preds = outputs.logits.squeeze().cpu().numpy()  
        predictions.extend(preds)

# Final test loss
avg_test_loss = test_loss / len(test_loader)
print(f'Test Loss: {avg_test_loss:.4f}')

test_true_labels = np.array(test_true_labels).flatten()
predictions = np.array(predictions)
print(test_true_labels.shape)
print(predictions.shape)

# Performance metrics
mse = mean_squared_error(test_true_labels, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(test_true_labels, predictions)
r2 = r2_score(test_true_labels, predictions)
PCC,_ = pearsonr(test_true_labels, predictions)
SCC,_ = spearmanr(test_true_labels, predictions)
# Print performance metrics
print(f'Mean Squared Error: {mse:.4f}')
print(f'Root Mean Squared Error: {rmse:.4f}')
print(f'Mean Absolute Error: {mae:.4f}')
print(f'R^2 Score: {r2:.4f}')
print(f'Pearson Correlation Coefficient: {PCC:.4f}')
print(f'Spearman Correlation Coefficient: {SCC:.4f}')

# Print hyperparameters
print("Hyperparameters:")
print(f"Learning Rate: {5e-5}")
print(f"Batch Size: 16")
print(f"Epochs: {num_epochs}")

Testing: 100%|██████████| 16/16 [00:02<00:00,  5.55batch/s]


Test Loss: 0.3105
(252,)
(252,)
Mean Squared Error: 0.3050
Root Mean Squared Error: 0.5523
Mean Absolute Error: 0.4132
R^2 Score: 0.4730
Pearson Correlation Coefficient: 0.7284
Spearman Correlation Coefficient: 0.7029
Hyperparameters:
Learning Rate: 5e-05
Batch Size: 16
Epochs: 20


In [11]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModel

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

model_name = 'ChemBERTa_model_1_caco2'
model_save_path = f'/kaggle/working/{model_name}'

if not os.path.exists(model_save_path):
    raise FileNotFoundError(f"The model directory {model_save_path} does not exist.")

tokenizer = AutoTokenizer.from_pretrained(model_save_path)
model = AutoModel.from_pretrained(model_save_path).to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at /kaggle/working/ChemBERTa_model_1_caco2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Load your datasets
train_df = pd.read_csv('/kaggle/input/caco2-set/Train_Caco2.csv')
train_df = train_df[['ID', 'SMILES', 'Permeability']]
test_df = pd.read_csv('/kaggle/input/caco2-set/Test_Caco2.csv')
test_df = test_df[['ID', 'SMILES', 'Permeability']]

In [13]:
train_encodings = tokenizer(list(train_df['SMILES']), truncation=True, padding=True, max_length=325, return_tensors="pt")
test_encodings = tokenizer(list(test_df['SMILES']), truncation=True, padding=True, max_length=325, return_tensors="pt")

In [14]:
from tqdm import tqdm 
batch_size = 16 

def generate_embeddings(encodings, batch_size):
    embeddings = []
    model.eval() 
    with torch.no_grad():
        for i in tqdm(range(0, len(encodings['input_ids']), batch_size), desc="Processing batches"):
            batch = {key: val[i:i + batch_size].to(device) for key, val in encodings.items()}  
            outputs = model(**batch)
            embeddings.append(outputs.last_hidden_state)
    return torch.cat(embeddings, dim=0)

In [15]:
train_embeddings = generate_embeddings(train_encodings, batch_size)
print(train_embeddings.shape)
train_embeddings = torch.mean(train_embeddings, dim=1)
print(train_embeddings.shape)

Processing batches: 100%|██████████| 63/63 [00:07<00:00,  8.31it/s]


torch.Size([1008, 216, 768])
torch.Size([1008, 768])


In [16]:
column_names = [f'x_fine_emb_ChemBerta{i}' for i in range(train_embeddings.shape[1])]
embeddings_df = pd.DataFrame(data=train_embeddings.cpu().numpy(), columns=column_names)
train_data = pd.concat([train_df, embeddings_df], axis=1)

In [17]:
test_embeddings = generate_embeddings(test_encodings, batch_size)
print(test_embeddings.shape)
test_embeddings = torch.mean(test_embeddings, dim=1)
print(test_embeddings.shape)

Processing batches: 100%|██████████| 16/16 [00:01<00:00,  8.78it/s]

torch.Size([252, 210, 768])
torch.Size([252, 768])


In [18]:
column_names = [f'x_fine_emb_ChemBerta{i}' for i in range(test_embeddings.shape[1])]
embeddings_df = pd.DataFrame(data=test_embeddings.cpu().numpy(), columns=column_names)
test_data = pd.concat([test_df, embeddings_df], axis=1)

In [19]:
train_data.to_csv("/kaggle/working/Train_ChemBERTa-zinc-base-v1_fine_tuned_embeddings_caco2.csv",index=False)
test_data.to_csv("/kaggle/working/Test_ChemBERTa-zinc-base-v1_fine_tuned_embeddings_caco2.csv",index=False)

In [20]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr, spearmanr
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [21]:
train_data = pd.read_csv("/kaggle/working/Train_ChemBERTa-zinc-base-v1_fine_tuned_embeddings_caco2.csv")
test_data = pd.read_csv("/kaggle/working/Test_ChemBERTa-zinc-base-v1_fine_tuned_embeddings_caco2.csv")

In [22]:
def train_and_test_predict(models, X_train, y_train, X_test, y_test):
    kf = KFold(n_splits=5, shuffle=True, random_state=101)
    results = {}
    predictions = []  

    for model in models:
        model_name = model.__class__.__name__
        predictions_train = []
        actual_y_train = []

        test_predictions_folds = []

        

        for train_index, val_index in kf.split(X_train):
            X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
            y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

            model.fit(X_train_fold, y_train_fold)

            y_pred_fold = model.predict(X_val_fold)
            y_pred_fold = np.clip(y_pred_fold, -10, -3.9)
            predictions_train.extend(y_pred_fold)
            actual_y_train.extend(y_val_fold)

            predictions_test_fold = model.predict(X_test)
            predictions_test_fold = np.clip(predictions_test_fold, -10, -3.9)
            test_predictions_folds.append(predictions_test_fold)


        mse_train = mean_squared_error(actual_y_train, predictions_train)
        mae_train = mean_absolute_error(actual_y_train, predictions_train)
        rmse_train = np.sqrt(mse_train)
        r2_train = r2_score(actual_y_train, predictions_train)
        pearson_train, _ = pearsonr(actual_y_train, predictions_train)
        spearman_train, _ = spearmanr(actual_y_train, predictions_train)


        predictions_test_mean = np.mean(test_predictions_folds, axis=0)
        predictions_test_std = np.std(test_predictions_folds, axis=0)

        mse_test = mean_squared_error(y_test, predictions_test_mean)
        mae_test = mean_absolute_error(y_test, predictions_test_mean)
        rmse_test = np.sqrt(mse_test)
        r2_test = r2_score(y_test, predictions_test_mean)
        print(r2_test)
        pearson_test, _ = pearsonr(y_test, predictions_test_mean)
        spearman_test, _ = spearmanr(y_test, predictions_test_mean)
        
        

        predictions.append({
            'Model': model_name,
            'Y Train pred': predictions_train,
            'Y Test actual': y_test,
            'Test prediction folds': test_predictions_folds,
            'Test Predictions Mean': predictions_test_mean,
            'Test Predictions Std': predictions_test_std,

        })

        results[model_name] = {
            'Train MSE (5 fold cv)': f"{mse_train:.4f}",
            'Train MAE (5 fold cv)': f"{mae_train:.4f}",
            'Train RMSE (5 fold cv)': f"{rmse_train:.4f}",
            'Train RMSE (5 fold cv)': f"{rmse_train:.4f}",
            'Train R2 (5 fold cv)': f"{r2_train:.4f}",
            'Train PCC (5 fold cv)': f"{pearson_train:.4f}",
            'Train SCC (5 fold cv)': f"{spearman_train:.4f}",
            'Test MSE': f"{mse_test:.4f}",
            'Test MAE': f"{mae_test:.4f}",
            'Test RMSE': f"{rmse_test:.4f}",
            'Test R2': f"{r2_test:.4f}",
            'Test Pearson Correlation': f"{pearson_test:.4f}",
            'Test Spearman Correlation': f"{spearman_test:.4f}",
        }

    results_df = pd.DataFrame(results).T
    predictions_df = pd.DataFrame(predictions)

    return results_df, predictions_df



In [23]:
X_train = train_data.drop(['ID','SMILES','Permeability'],axis=1)
y_train = train_data['Permeability']
print("X_train shape: ",X_train.shape)
print("y_train shape: ",y_train.shape)
print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")

X_test = test_data.drop(['ID','SMILES','Permeability'],axis=1)
y_test = test_data['Permeability']
print("X_test shape: ",X_test.shape)
print("y_test shape: ",y_test.shape)
print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)
models = [
    lgb.LGBMRegressor(objective='regression',metric='rmse',boosting_type='gbdt',num_leaves=31,learning_rate=0.05,random_state=101),
    DecisionTreeRegressor(random_state=101),
    RandomForestRegressor(n_jobs=-1, random_state=101),
    GradientBoostingRegressor(random_state=101),
    AdaBoostRegressor(random_state=101),
    xgb.XGBRegressor(random_state=101),
    ExtraTreesRegressor(n_jobs=-1, n_estimators=100, random_state=101),
    LinearRegression(), 
    KNeighborsRegressor(n_neighbors=3),
    SVR(),  
    MLPRegressor(random_state=101)
]
result_df, prediction_df = train_and_test_predict(models, X_train,y_train, X_test,  y_test)
result_df

X_train shape:  (1008, 768)
y_train shape:  (1008,)
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
X_test shape:  (252, 768)
y_test shape:  (252,)
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009510 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 195840
[LightGBM] [Info] Number of data points in the train set: 806, number of used features: 768
[LightGBM] [Info] Start training from score -6.296846
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furth

,Train MSE (5 fold cv),Train MAE (5 fold cv),Train RMSE (5 fold cv),Train R2 (5 fold cv),Train PCC (5 fold cv),Train SCC (5 fold cv),Test MSE,Test MAE,Test RMSE,Test R2,Test Pearson Correlation,Test Spearman Correlation
LGBMRegressor,0.0578,0.1766,0.2404,0.9047,0.9512,0.9381,0.2790,0.3946,0.5282,0.5179,0.7312,0.7090
DecisionTreeRegressor,0.1109,0.2501,0.3330,0.8172,0.9078,0.8869,0.2828,0.4026,0.5318,0.5113,0.7290,0.7069
RandomForestRegressor,0.0560,0.1744,0.2366,0.9077,0.9528,0.9392,0.2806,0.3962,0.5297,0.5151,0.7279,0.7080
GradientBoostingRegressor,0.0572,0.1769,0.2393,0.9056,0.9516,0.9379,0.2768,0.3915,0.5261,0.5218,0.7327,0.7111
AdaBoostRegressor,0.0615,0.1870,0.2480,0.8986,0.9486,0.9358,0.2741,0.3948,0.5236,0.5263,0.7306,0.7035
XGBRegressor,0.0689,0.1920,0.2626,0.8863,0.9415,0.9273,0.2788,0.4008,0.5280,0.5183,0.7296,0.7066
ExtraTreesRegressor,0.0585,0.1751,0.2420,0.9034,0.9505,0.9382,0.2769,0.3972,0.5262,0.5216,0.7315,0.7097
LinearRegression,2.2865,1.1810,1.5121,-2.7709,0.4015,0.3900,0.9441,0.7144,0.9716,-0.6313,0.4637,0.4508
KNeighborsRegressor,0.0732,0.1983,0.2705,0.8793,0.9381,0.9186,0.2687,0.3857,0.5183,0.5357,0.7436,0.7121
SVR,0.0544,0.1699,0.2333,0.9103,0.9541,0.9431,0.2753,0.3952,0.5247,0.5242,0.7345,0.7105


In [24]:
result_df

,Train MSE (5 fold cv),Train MAE (5 fold cv),Train RMSE (5 fold cv),Train R2 (5 fold cv),Train PCC (5 fold cv),Train SCC (5 fold cv),Test MSE,Test MAE,Test RMSE,Test R2,Test Pearson Correlation,Test Spearman Correlation
LGBMRegressor,0.0578,0.1766,0.2404,0.9047,0.9512,0.9381,0.2790,0.3946,0.5282,0.5179,0.7312,0.7090
DecisionTreeRegressor,0.1109,0.2501,0.3330,0.8172,0.9078,0.8869,0.2828,0.4026,0.5318,0.5113,0.7290,0.7069
RandomForestRegressor,0.0560,0.1744,0.2366,0.9077,0.9528,0.9392,0.2806,0.3962,0.5297,0.5151,0.7279,0.7080
GradientBoostingRegressor,0.0572,0.1769,0.2393,0.9056,0.9516,0.9379,0.2768,0.3915,0.5261,0.5218,0.7327,0.7111
AdaBoostRegressor,0.0615,0.1870,0.2480,0.8986,0.9486,0.9358,0.2741,0.3948,0.5236,0.5263,0.7306,0.7035
XGBRegressor,0.0689,0.1920,0.2626,0.8863,0.9415,0.9273,0.2788,0.4008,0.5280,0.5183,0.7296,0.7066
ExtraTreesRegressor,0.0585,0.1751,0.2420,0.9034,0.9505,0.9382,0.2769,0.3972,0.5262,0.5216,0.7315,0.7097
LinearRegression,2.2865,1.1810,1.5121,-2.7709,0.4015,0.3900,0.9441,0.7144,0.9716,-0.6313,0.4637,0.4508
KNeighborsRegressor,0.0732,0.1983,0.2705,0.8793,0.9381,0.9186,0.2687,0.3857,0.5183,0.5357,0.7436,0.7121
SVR,0.0544,0.1699,0.2333,0.9103,0.9541,0.9431,0.2753,0.3952,0.5247,0.5242,0.7345,0.7105


In [25]:
prediction_df

,Model,Y Train pred,Y Test actual,Test prediction folds,Test Predictions Mean,Test Predictions Std
0,LGBMRegressor,"[-7.292462681315174, -7.5816599934724875, -6.9...",0 -7.19 1 -6.21 2 -7.24 3 -5.8...,"[[-7.02498166841009, -5.677531556147479, -5.73...","[-7.011098290154813, -5.677657866208772, -5.75...","[0.03599470284888139, 0.05166373952587775, 0.0..."
1,DecisionTreeRegressor,"[-7.41, -7.03, -7.22, -6.68, -6.28, -6.92, -5....",0 -7.19 1 -6.21 2 -7.24 3 -5.8...,"[[-7.0, -5.59, -5.96, -5.92, -5.54, -6.8, -5.9...","[-6.922774392799999, -5.696, -5.81464565319999...","[0.15095325824120112, 0.13169662106523464, 0.1..."
2,RandomForestRegressor,"[-7.237957574910001, -7.663358800159997, -7.02...",0 -7.19 1 -6.21 2 -7.24 3 -5.8...,"[[-7.02289389292, -5.709211846339998, -5.78094...","[-7.000684719968001, -5.69114922889, -5.798257...","[0.02774833928049578, 0.021328592025846155, 0...."
3,GradientBoostingRegressor,"[-7.319278610307436, -7.551383576162681, -6.98...",0 -7.19 1 -6.21 2 -7.24 3 -5.8...,"[[-7.0464238051456505, -5.721889441915605, -5....","[-7.0037229190259875, -5.73188943775403, -5.76...","[0.04674952409970827, 0.044425233947629844, 0...."
4,AdaBoostRegressor,"[-7.2, -7.498999999999999, -7.024264136171874,...",0 -7.19 1 -6.21 2 -7.24 3 -5.8...,"[[-6.992817969630868, -5.745020350880184, -5.7...","[-6.942262490403431, -5.691302683806768, -5.77...","[0.06162240980432033, 0.04679456505195146, 0.0..."
5,XGBRegressor,"[-7.336082, -7.9073353, -6.917879, -6.8506694,...",0 -7.19 1 -6.21 2 -7.24 3 -5.8...,"[[-7.1687274, -5.77472, -5.7993746, -5.907785,...","[-6.984561, -5.6831555, -5.81688, -5.854247, -...","[0.12944558, 0.052518845, 0.094338775, 0.08323..."
6,ExtraTreesRegressor,"[-7.294226458880003, -7.6546999999999965, -6.9...",0 -7.19 1 -6.21 2 -7.24 3 -5.8...,"[[-7.0098213972600005, -5.720984778560002, -5....","[-6.971507747422, -5.681087192438001, -5.82335...","[0.02419666324639479, 0.027388123899037758, 0...."
7,LinearRegression,"[-5.16117284066149, -7.175818078129848, -7.423...",0 -7.19 1 -6.21 2 -7.24 3 -5.8...,"[[-6.372213239724258, -8.533323731932793, -5.2...","[-7.450501750506096, -6.2072584716049475, -5.2...","[0.7529879090222406, 2.114482528387649, 0.6371..."
8,KNeighborsRegressor,"[-7.426666666666667, -7.343333333333334, -6.98...",0 -7.19 1 -6.21 2 -7.24 3 -5.8...,"[[-7.109999999999999, -5.650000000000001, -5.8...","[-7.090666666666668, -5.6673333333333344, -5.8...","[0.028782710859897104, 0.023701851779507684, 0..."
9,SVR,"[-7.138178023895527, -7.611015940881996, -6.91...",0 -7.19 1 -6.21 2 -7.24 3 -5.8...,"[[-7.031055314513305, -5.714109343336877, -5.8...","[-6.9892732549961405, -5.7255970911396705, -5....","[0.03461277241312399, 0.02153458392142625, 0.0..."


In [26]:
result_df.to_csv('/kaggle/working/Results_ChemBERTa-zinc-base-v1_fine_tuned_embeddings_caco2.csv')
prediction_df.to_csv('/kaggle/working/Prediction_data_ChemBERTa-zinc-base-v1_fine_tuned_embeddings_caco2.csv')

In [29]:
result_df

,Train MSE (5 fold cv),Train MAE (5 fold cv),Train RMSE (5 fold cv),Train R2 (5 fold cv),Train PCC (5 fold cv),Train SCC (5 fold cv),Test MSE,Test MAE,Test RMSE,Test R2,Test Pearson Correlation,Test Spearman Correlation
LGBMRegressor,0.0578,0.1766,0.2404,0.9047,0.9512,0.9381,0.2790,0.3946,0.5282,0.5179,0.7312,0.7090
DecisionTreeRegressor,0.1109,0.2501,0.3330,0.8172,0.9078,0.8869,0.2828,0.4026,0.5318,0.5113,0.7290,0.7069
RandomForestRegressor,0.0560,0.1744,0.2366,0.9077,0.9528,0.9392,0.2806,0.3962,0.5297,0.5151,0.7279,0.7080
GradientBoostingRegressor,0.0572,0.1769,0.2393,0.9056,0.9516,0.9379,0.2768,0.3915,0.5261,0.5218,0.7327,0.7111
AdaBoostRegressor,0.0615,0.1870,0.2480,0.8986,0.9486,0.9358,0.2741,0.3948,0.5236,0.5263,0.7306,0.7035
XGBRegressor,0.0689,0.1920,0.2626,0.8863,0.9415,0.9273,0.2788,0.4008,0.5280,0.5183,0.7296,0.7066
ExtraTreesRegressor,0.0585,0.1751,0.2420,0.9034,0.9505,0.9382,0.2769,0.3972,0.5262,0.5216,0.7315,0.7097
LinearRegression,2.2865,1.1810,1.5121,-2.7709,0.4015,0.3900,0.9441,0.7144,0.9716,-0.6313,0.4637,0.4508
KNeighborsRegressor,0.0732,0.1983,0.2705,0.8793,0.9381,0.9186,0.2687,0.3857,0.5183,0.5357,0.7436,0.7121
SVR,0.0544,0.1699,0.2333,0.9103,0.9541,0.9431,0.2753,0.3952,0.5247,0.5242,0.7345,0.7105


In [28]:
prediction_df

,Model,Y Train pred,Y Test actual,Test prediction folds,Test Predictions Mean,Test Predictions Std
0,LGBMRegressor,"[-7.292462681315174, -7.5816599934724875, -6.9...",0 -7.19 1 -6.21 2 -7.24 3 -5.8...,"[[-7.02498166841009, -5.677531556147479, -5.73...","[-7.011098290154813, -5.677657866208772, -5.75...","[0.03599470284888139, 0.05166373952587775, 0.0..."
1,DecisionTreeRegressor,"[-7.41, -7.03, -7.22, -6.68, -6.28, -6.92, -5....",0 -7.19 1 -6.21 2 -7.24 3 -5.8...,"[[-7.0, -5.59, -5.96, -5.92, -5.54, -6.8, -5.9...","[-6.922774392799999, -5.696, -5.81464565319999...","[0.15095325824120112, 0.13169662106523464, 0.1..."
2,RandomForestRegressor,"[-7.237957574910001, -7.663358800159997, -7.02...",0 -7.19 1 -6.21 2 -7.24 3 -5.8...,"[[-7.02289389292, -5.709211846339998, -5.78094...","[-7.000684719968001, -5.69114922889, -5.798257...","[0.02774833928049578, 0.021328592025846155, 0...."
3,GradientBoostingRegressor,"[-7.319278610307436, -7.551383576162681, -6.98...",0 -7.19 1 -6.21 2 -7.24 3 -5.8...,"[[-7.0464238051456505, -5.721889441915605, -5....","[-7.0037229190259875, -5.73188943775403, -5.76...","[0.04674952409970827, 0.044425233947629844, 0...."
4,AdaBoostRegressor,"[-7.2, -7.498999999999999, -7.024264136171874,...",0 -7.19 1 -6.21 2 -7.24 3 -5.8...,"[[-6.992817969630868, -5.745020350880184, -5.7...","[-6.942262490403431, -5.691302683806768, -5.77...","[0.06162240980432033, 0.04679456505195146, 0.0..."
5,XGBRegressor,"[-7.336082, -7.9073353, -6.917879, -6.8506694,...",0 -7.19 1 -6.21 2 -7.24 3 -5.8...,"[[-7.1687274, -5.77472, -5.7993746, -5.907785,...","[-6.984561, -5.6831555, -5.81688, -5.854247, -...","[0.12944558, 0.052518845, 0.094338775, 0.08323..."
6,ExtraTreesRegressor,"[-7.294226458880003, -7.6546999999999965, -6.9...",0 -7.19 1 -6.21 2 -7.24 3 -5.8...,"[[-7.0098213972600005, -5.720984778560002, -5....","[-6.971507747422, -5.681087192438001, -5.82335...","[0.02419666324639479, 0.027388123899037758, 0...."
7,LinearRegression,"[-5.16117284066149, -7.175818078129848, -7.423...",0 -7.19 1 -6.21 2 -7.24 3 -5.8...,"[[-6.372213239724258, -8.533323731932793, -5.2...","[-7.450501750506096, -6.2072584716049475, -5.2...","[0.7529879090222406, 2.114482528387649, 0.6371..."
8,KNeighborsRegressor,"[-7.426666666666667, -7.343333333333334, -6.98...",0 -7.19 1 -6.21 2 -7.24 3 -5.8...,"[[-7.109999999999999, -5.650000000000001, -5.8...","[-7.090666666666668, -5.6673333333333344, -5.8...","[0.028782710859897104, 0.023701851779507684, 0..."
9,SVR,"[-7.138178023895527, -7.611015940881996, -6.91...",0 -7.19 1 -6.21 2 -7.24 3 -5.8...,"[[-7.031055314513305, -5.714109343336877, -5.8...","[-6.9892732549961405, -5.7255970911396705, -5....","[0.03461277241312399, 0.02153458392142625, 0.0..."
